<a href="https://colab.research.google.com/github/dinael2000/MDE_to_3D/blob/main/run_pipeline/full_workflow_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Leveraging Monocular Depth Estimation for the rapid 3D Digitization of Cultural Heritage Artefacts 🤖

## 💻 Full MDE-to-3D Pipeline

The following Notebook provides a full pipeline for the production of 3D models of coin-like objects, starting from a single image as input.

It is being developed as part of an ongoing Master Thesis on:
"Utilizing Monocular Depth Estimation (MDE) Models for the Digitization of Cultural Heritage Objects. Leveraging deep learning-based MDE-Algorithms for the production of 3D models of archaeological artefacts that carry micro-reliefs.", carried out at the Universität zu Köln.  

Make sure to go through all cells sequentially. Click the start button (▶) next to each one to get it to work.

### 1. Setting up the environment

In [ ]:
%%capture
# 1.1. Clone github repository
!git clone https://github.com/dinael2000/MDE_to_3D.git

In [ ]:
# 1.2.1. Install required libraries

%cd /content/MDE_to_3D

!pip install -r requirements.txt

# Click on the downwards pointing arrowhead to collapse the output table

This will probably take a few minutes, so don't stress it 😊

❗When prompted to, click on **Restart Session** and continue to the next cell❗

In [ ]:
# 1.2.2. Re-run after restarting session
%%capture
!pip install -r requirements.txt
!pip install open3d

In [ ]:
# 1.3. Initiate libraries
%%capture
import os

In [ ]:
# 1.4. Read project modules
%%capture
%cd /content/MDE_to_3D/run_pipeline

from depthanythingv1_function import *
from openback import *
from watertight import *
from merge_meshes import *
from scale_meshes import *
from utils_colors import *
from utils_pointcloud import *

### 2. Set up depth estimation pipeline

In [ ]:
# 2.1. Initiate directories

# Directory with input images
rgb_dir = "rgb_dir"
os.makedirs(rgb_dir, exist_ok=True)

# Directory to save .npy depth maps
depth_npy_dir = "depth_npy_dir"
os.makedirs(depth_npy_dir, exist_ok=True)

# Directory to save colored depth maps
depth_colored_dir = "depth_colored_dir"
os.makedirs(depth_colored_dir, exist_ok=True)

# Directory to save black-and-white depth maps
depth_bw_dir = "depth_bw_dir"
os.makedirs(depth_bw_dir, exist_ok=True)

# 2.2. Choose colormap-scheme for colored depth map

In [ ]:
color_scheme = "inferno" # @param ["inferno","viridis","plasma","magma","cividis"] {"allow-input":true}

In [ ]:
# 2.3 Upload images
from google.colab import files

%cd /content/MDE_to_3D/run_pipeline/rgb_dir
uploads = files.upload()

%cd /content/MDE_to_3D/run_pipeline

This will also take some time.

For quicker processing times, you may also upload the desired file(s) directly to:
/content/MDE_to_3D/run_pipeline/rgb_dir
(CNTRL + Left Click for quick access)

❗ Important Notes ❗

* Make sure that the object is mostly **centered** in the frame of the uploaded picture
* Make sure that the object occupies a **large space** in the frame of the uploaded picture
* Should you wish to *merge* two sides of an object (obv-rev), make sure that:
1. Each side occupies roughly the same space in each frame (to facilitate alignment)
2. Each side is named appropriately (e.g. "{object_name}-obv", "{object_name}-rev"






### 3. Depth Estimation with DepthAnything V1
Yang, L., Kang, B., Huang, Z., Xu, X., Feng, J., Zhao, H., "Depth Anything: Unleashing the Power of Large-Scale Unlabeled Data". 2024.

Available at: [GitHub](https://github.com/globalwetlands/depth-anything-V1), [HuggingFace](https://huggingface.co/spaces/LiheYoung/Depth-Anything)

In [ ]:
# Run depth estimation pipeline

run_depth_estimation(rgb_dir=rgb_dir, depth_npy_dir=depth_npy_dir, depth_bw_dir=depth_bw_dir, depth_colored_dir=depth_colored_dir, color_scheme=color_scheme)

### 4. 3D model creation

The following sections provide different options for creating 3D models, using the depth maps produced in the previous steps:

1. The *first* section enables the production of non-watertight models, which can be used as such, or to then merge into a single model.

2. The *second* section production the creation of (mostly) watertight models, which can be used as such.

3. The *third* section enables merging two (non-watertight) models into a single, full object.

4. The *fourth* section enables scaling any given model.



### 4.1. Set up document-wide parameters

See *Parameter Index* for more information.

To change, simply adjust the value and re-run the code.

In [ ]:
pixel_size = 1 # @param {"type":"slider","min":0.5,"max":1,"step":0.1}


(**Recommended Value**: 1.0)

In [ ]:
relief_scale = 8 # @param {"type":"slider","min":0,"max":10,"step":1}


(**Recommended Value**: 8.0)

In [ ]:
poisson_depth = 9 # @param {"type":"slider","min":5,"max":10,"step":1}


(**Recommended Value**: 9.0)

In [ ]:
output_format = "obj" # @param ["obj","ply"] {"allow-input":true}


(**Recommended Value**: "obj")

In [ ]:
gap_factor = -0.5 # @param {"type":"slider","min":-1,"max":1,"step":0.1}


**Recommended Value**: -0.5 (Works with *most* but *not all* tested objects)

#### 4.2. Non-Watertight Models

In [ ]:
# 4.1.1. Set up directories
out_dir_openback = "3d_models_openback"
os.makedirs(out_dir_openback, exist_ok=True)

In [ ]:
# 4.1.2. Run module
batch_process_openback(depth_npy_dir, rgb_dir, out_dir_openback, output_format=output_format, pixel_size=pixel_size, relief_scale=relief_scale, poisson_depth=poisson_depth)

In [ ]:
# 4.1.3. Download created models
from google.colab import files

path = "/content/MDE_to_3D/run_pipeline/3d_models_openback"

for mesh in os.listdir(path):
  file_path = os.path.join(path, mesh)
  print(file_path)
  files.download(file_path)

#### 4.3. Watertight Models

This module takes a significant amount of time to run. It is recommended that you process a smaller dataset.

Or grab a coffee while waiting ☕



In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
# 4.2.1. Set up directories
out_dir_watertight = "3d_models_watertight"
os.makedirs(out_dir_watertight, exist_ok=True)

In [ ]:
# 4.2.2. Run module
batch_process_watertight(depth_npy_dir, rgb_dir, out_dir_watertight, output_format=output_format, pixel_size=pixel_size, relief_scale=relief_scale, poisson_depth=poisson_depth)

In [ ]:
# 4.2.3. Download created models
from google.colab import files

path = "/content/MDE_to_3D/run_pipeline/3d_models_watertight"

for mesh in os.listdir(path):
  file_path = os.path.join(path, mesh)
  print(file_path)
  files.download(file_path)

#### 4.4. Merged Models

In [ ]:
# 4.3.1. Set up directories
input_dir = "/content/MDE_to_3D/run_pipeline/3d_models_openback"
output_dir_merged = "3d_models_merged"
os.makedirs(output_dir_merged, exist_ok=True)

In [ ]:
# 4.3.2. Run module
batch_process_merge(input_dir, output_dir_merged, gap_factor=gap_factor, scale=False, scale_factor=0)

In [ ]:
# 4.3.3. Download created models
from google.colab import files

path = "/content/MDE_to_3D/run_pipeline/3d_models_merged"

for mesh in os.listdir(path):
  file_path = os.path.join(path, mesh)
  print(file_path)
  files.download(file_path)

#### 4.4. Scaled Models

In [ ]:
# 4.4.1. Set up directories

In [ ]:
input_dir_to_scale = "/content/MDE_to_3D/run_pipeline/3d_models_merged" # @param ["/content/MDE_to_3D/run_pipeline/3d_models_openback","/content/MDE_to_3D/run_pipeline/3d_models_watertight","/content/MDE_to_3D/run_pipeline/3d_models_merged"]


In [ ]:
output_dir_scaled = "3d_models_scaled"
os.makedirs(output_dir_scaled, exist_ok=True)

In [ ]:
# 4.4.2. Set up scale factor

In [ ]:
scale_factor = 0 # @param {"type":"slider","min":0,"max":1,"step":0.01}


In [ ]:
# 4.4.3. Run module
batch_scale_models(input_dir_to_scale, output_dir_scaled, scale_factor=scale_factor)

In [ ]:
# 4.4.4. Download created models
from google.colab import files
path = "/content/MDE_to_3D/run_pipeline/3d_models_scaled"

for mesh in os.listdir(path):
  file_path = os.path.join(path, mesh)
  print(file_path)
  files.download(file_path)

In [ ]:
import plotly.graph_objects as go
import numpy as np
import open3d as o3d

In [ ]:
mesh_folder = "/content/MDE_to_3D/run_pipeline/3d_models_merged" # @param ["/content/MDE_to_3D/run_pipeline/3d_models_openback","/content/MDE_to_3D/run_pipeline/3d_models_watertight","/content/MDE_to_3D/run_pipeline/3d_models_merged"]


#### ❗ Parameter Index ❗:
##### Setting up parameters
In order to run the 3D-model generation modules, certain parameters have to be set up. These are the following:
1. *pixel_size*:

Pixel Size controls the XY spacing between neighboring depth samples, setting the real-world scale per pixel in the X and Y directions. This parameter affects the overall size of the reconstructed object, *without* affecting shape.

A **smaller** pixel_size value shrinks the object in XY, permitting higher detail density. A **larger** pixel_size value increases the object in XY, making the surface less detailed.

**Recommended Range**: 0.5-1.0

**Recommended Value**: 1.0

2. *relief_scale*

Relief Scale controls depth variation (Z) relative to X and Y. By scaling the object surface, it accounts for the low dynamic range present in MDE-created depth maps.

A **smaller** relief_scale smoothens surface-details on the object. A **larger** relief_scale exaggerates surface-details on the object.

**Recommended Range**: 5.0-9.0

**Recommended Value**: 8.0

3. *poisson_depth*

Poisson Octree Depth controls the model's resolution, by setting the max subdivision level for the reconstruction triangle count and surface detail.  

A **smaller** poisson_depth results in a coarser model, but requires less computation time. A **larger** poisson_depth results in a more detailed model, but requires more computation time. A **very large** poisson_depth may result in a lot of unecessary noise.

**Recommended Range**: 7-10

**Recommended Value**: 8 or 9

4. *output_format*

Output format controls the file-type for the generated 3D model.

**Recommended Options**: "obj", "ply"

**Recommended Value**: "obj"

5. *gap_factor*

Gap factor controls the gap in the Z axis that will be allowed between each aligned side of the model-to-be-merged.

A **smaller** gap_factor brings the two sides closer together, but may result in conflicting geometry. A **larger** gap_factor seperates the two sides further, but may result in gaps in the mesh. Requires trial-and-error, can be decided on a case-by-case scenario.

**Recommended Range**: -0.5-1.0

**Recommended Value**: -0.5 (Works with *most* but *not all* tested objects)

6. *scale_factor*

Scale factor controls the value by which the object will be scaled.
